## <b><font color='darkblue'>Preface</font></b>
([article source](https://realpython.com/build-llm-rag-chatbot-with-langchain/)) <b><font size='3ptx'>You’ve likely interacted with large language models (LLMs), like the ones behind OpenAI’s ChatGPT, and experienced their remarkable ability to answer questions, summarize documents, write code, and much more. While LLMs are remarkable by themselves, with a little programming knowledge, you can leverage libraries like [LangChain](https://python.langchain.com/docs/get_started/introduction) to create your own LLM-powered chatbots that can do just about anything.</font></b>

<b>In an enterprise setting, one of the most popular ways to create an LLM-powered chatbot is through [retrieval-augmented generation](https://www.promptingguide.ai/techniques/rag)</b> (<font color='brown'>RAG</font>). When you design a RAG system, you use a retrieval model to retrieve relevant information, usually from a database or corpus, and provide this retrieved information to an LLM to generate contextually relevant responses.

<b>In this tutorial, you’ll step into the shoes of an AI engineer working for a large hospital system. You’ll build a RAG chatbot in LangChain that uses Neo4j to retrieve data about the patients, patient experiences, hospital locations, visits, insurance payers, and physicians in your hospital system</b>.

In this tutorial, you’ll learn how to:
* Use **LangChain** to build custom **chatbots**
* Design a **chatbot using your understanding of the business requirements and hospital system data**.
* Work with **graph databases**.
* Set up a Neo4j AuraDB instance.
* **Build a RAG chatbot** that retrieves both structured and unstructured data from Neo4j
* **Deploy your chatbot** with **FastAPI** and **Streamlit**.

## <b><font color='darkblue'>Prerequisites</font></b>
This tutorial is best suited for intermediate Python developers who want to get hands-on experience creating custom chatbots. Aside from intermediate Python knowledge, you’ll benefit from having a high-level understanding of the following concepts and technologies:
* Large language models (LLMs) and [prompt engineering](https://realpython.com/practical-prompt-engineering/).
* [Text embeddings and vector databases](https://realpython.com/chromadb-vector-database/#represent-data-as-vectors).
* [Graph databases](https://neo4j.com/developer/graph-database/) and [Neo4j](https://neo4j.com/docs/getting-started/languages-guides/neo4j-python/).
* [The OpenAI developer ecosystem](https://openai.com/product).
* [REST APIs](https://realpython.com/api-integration-in-python/) and [FastAPI](https://realpython.com/fastapi-python-web-apis/).
* Asynchronous programming
* Docker and Docker Compose

Nothing listed above is a hard prerequisite, so don’t worry if you don’t feel knowledgeable in any of them. You’ll be introduced to each concept and technology along the way. Besides, there’s no better way to learn these prerequisites than to implement them yourself in this tutorial.

Next up, you’ll get a brief project overview and begin learning about LangChain.

## <b><font color='darkblue'>Project Overview</font></b>
Throughout this tutorial, you’ll create a few directories that make up your final chatbot. Here’s a breakdown of each directory:
* **<font color='#556B2F'>langchain_intro/</font>** will help you get familiar with LangChain and equip you with the tools that you need to build the chatbot you saw in the demo, and it won’t be included in your final chatbot. You’ll cover this in [Step 1](#step1).
* **<font color='#556B2F'>data/</font>** has the raw hospital system data stored as CSV files. You’ll explore this data in [Step 2](https://realpython.com/build-llm-rag-chatbot-with-langchain/#step-2-understand-the-business-requirements-and-data). In [Step 3](https://realpython.com/build-llm-rag-chatbot-with-langchain/#step-3-set-up-a-neo4j-graph-database), you’ll move this data into a Neo4j database that your chatbot will query to answer questions.
* **<font color='#556B2F'>hospital_neo4j_etl/</font>** contains a script that loads the raw data from <font color='#556B2F'>data/</font> into your Neo4j database. You have to run this before building your chatbot, and you’ll learn everything you need to know about setting up a Neo4j instance in Step 3.
* **<font color='#556B2F'>chatbot_api/</font>** is your [FastAPI](https://realpython.com/fastapi-python-web-apis/) app that serves your chatbot as a REST endpoint, and it’s the core deliverable of this project. The <font color='#556B2F'>`chatbot_api/src/agents/`</font> and <font color='#556B2F'>`chatbot_api/src/chains/`</font> subdirectories contain the LangChain objects that comprise your chatbot. You’ll learn what agents and chains are later, but for now, just know that your chatbot is actually a LangChain agent composed of chains and functions.
* **<font color='#556B2F'>tests/</font>** includes two scripts that test how fast your chatbot can answer a series of questions. This will give you a feel for how much time you save by making asynchronous requests to LLM providers like OpenAI.
* **<font color='#556B2F'>chatbot_frontend/</font>** is your Streamlit app that interacts with the chatbot endpoint in <font color='#556B2F'>chatbot_api/</font>. This is the UI that you saw in the demo, and you’ll build this in [Step 5](https://realpython.com/build-llm-rag-chatbot-with-langchain/#step-5-deploy-the-langchain-agent).

All the environment variables needed to build and run your chatbot will be stored in a .env file. You’ll deploy the code in <font color='#556B2F'>hospital_neo4j_etl/</font>, `chatbot_api`, and `chatbot_frontend` as Docker containers that’ll be orchestrated with Docker Compose. If you want to experiment with the chatbot before going through the rest of this tutorial, then you can download the materials and follow the instructions in the README file to get things running:
> Get Your Code: [Click here to download the free source code](https://realpython.com/bonus/build-llm-rag-chatbot-with-langchain-code/) for your LangChain chatbot.

<br/>
With the project overview and prerequisites behind you, you’re ready to get started with the first step—getting familiar with LangChain.

<a id='agenda'></a>
### <b><font color='darkgreen'>Agenda</font></b>
* <font size='3ptx'><b><a href='#step1'>Step 1: Get Familiar With LangChain</a></b></font>
* <font size='3ptx'><b><a href='#step2'>Step 2: Understand the Business Requirements and Data</a></b></font>
* <font size='3ptx'><b><a href='#step3'>Step 3: Set Up a Neo4j Graph Database</a></b></font>
* <font size='3ptx'><b><a href='#step4'>Step 4: Build a Graph RAG Chatbot in LangChain</a></b></font>

<a id='step1'></a>
## <b><font color='darkblue'>Step 1: Get Familiar With LangChain</font></b>
* <font size='3ptx'><b><a href='#step1_1'>Chat Models</a></b></font>
* <font size='3ptx'><b><a href='#step1_2'>Prompt Templates</a></b></font>
* <font size='3ptx'><b><a href='#step1_3'>Chains and LangChain Expression Language (LCEL)</a></b></font>
* <font size='3ptx'><b><a href='#step1_4'>Retrieval Objects</a></b></font>
* <font size='3ptx'><b><a href='#step1_5'>Agents</a></b></font>

<b><font size='3ptx'>Before you design and develop your chatbot, you need to know how to use [LangChain](https://github.com/langchain-ai/langchain). </font></b>

<b>In this section, you’ll get to know LangChain’s main components and features by building a preliminary version of your hospital system chatbot</b>. This will give you all the necessary tools to build your full chatbot.

Use your favorite code editor to create a new Python project, and be sure to <b>create a virtual environment for its dependencies. Make sure you have Python 3.10 or later installed. Activate your virtual environment and install the following libraries</b>:

In [1]:
# !python -m pip install langchain==0.1.0 openai==1.7.2 langchain-openai==0.0.2 langchain-community==0.0.12 langchainhub==0.1.14

You’ll also want to install [**python-dotenv**](https://pypi.org/project/python-dotenv/) to help you manage environment variables:

In [2]:
#!python -m pip install python-dotenv

**[Python-dotenv](https://pypi.org/project/python-dotenv/) loads environment variables from `.env` files into your Python environment, and you’ll find this handy as you develop your chatbot**. However, you’ll eventually deploy your chatbot with Docker, which can handle environment variables for you, and you won’t need Python-dotenv anymore.

If you haven’t already, you’ll need to download <font color='olive'>reviews.csv</font> from the materials or [GitHub repo](https://github.com/hfhoffman1144/langchain_neo4j_rag_app/blob/main/data/reviews.csv) for this tutorial:
> Get Your Code: [Click here to download the free source code](https://realpython.com/bonus/build-llm-rag-chatbot-with-langchain-code/) for your LangChain chatbot.

<br/>

Next, open the project directory and add the following folders and files:
```
./
│
├── data/
│   └── reviews.csv
│
├── langchain_intro/
│   ├── chatbot.py
│   ├── create_retriever.py
│   └── tools.py
│
└── .env
```

<br/>

The <font color='olive'>reviews.csv</font> file in <font color='olive'>data/</font> is the one you just downloaded, and the remaining files you see should be empty. <b>You’re now ready to get started building your first chatbot with LangChain!</b>

<a id='step1_1'></a>
### <b><font color='darkgreen'>Chat Models</font></b>
<b><font size='3ptx'>You might’ve guessed that the core component of LangChain is the [LLM](https://python.langchain.com/docs/modules/model_io/llms/). </font></b>

<b>LangChain provides a modular interface for working with LLM providers such as OpenAI, Cohere, HuggingFace, Anthropic, Together AI, and others</b>. In most cases, all you need is an API key from the LLM provider to get started using the LLM with LangChain. LangChain also supports LLMs or other language models hosted on your own machine.

<b>You’ll use OpenAI for this tutorial, but keep in mind there are many great open- and closed-source providers out there</b>. You can always test out different providers and optimize depending on your application’s needs and cost constraints. Before moving forward, make sure you’re signed up for an OpenAI account and you have a valid [**API key**](https://openai.com/product).

Once you have your OpenAI API key, add it to your `.env` file:
```
OPENAI_API_KEY=<YOUR-OPENAI-API-KEY>
```

<br/>

While you can interact directly with LLM objects in LangChain, a more common abstraction is the [**chat model**](https://python.langchain.com/docs/modules/model_io/chat/). <b>Chat models use LLMs under the hood, but they’re designed for conversations, and they interface with chat messages rather than [raw text](https://python.langchain.com/docs/modules/model_io/chat/quick_start#messages)</b>.

<b>Using chat messages, you provide an LLM with additional detail about the kind of message you’re sending. All messages have `role` and `content` properties</b>. The role tells the LLM who is sending the message, and the content is the message itself. Here are the most commonly used messages:
* [**HumanMessage**](https://python.langchain.com/docs/modules/model_io/concepts#humanmessage): A message from the user interacting with the language model.
* [**AIMessage**](https://python.langchain.com/docs/modules/model_io/concepts#aimessage): A message from the language model.
* [**SystemMessage**](https://python.langchain.com/docs/modules/model_io/concepts#systemmessage): A message that tells the language model how to behave. Not all providers support the [**SystemMessage**](https://python.langchain.com/docs/modules/model_io/concepts#systemmessage).

There are other messages types, like [**FunctionMessage**](https://python.langchain.com/docs/modules/model_io/concepts#functionmessage) and [**ToolMessage**](https://python.langchain.com/docs/modules/model_io/concepts#toolmessage), but you’ll learn more about those when you build an [**agent**](https://python.langchain.com/docs/modules/agents/).

Getting started with chat models in LangChain is straightforward. <b>To instantiate an OpenAI chat model, navigate to <font color='olive'>langchain_intro</font> and add the following code to <font color='olive'>chatbot.py</font></b>:
```python
"""Chat bot module."""
import dotenv
from langchain_openai import ChatOpenAI


dotenv.load_dotenv()
chat_model = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)
```

<br/>

You first <font color='blue'>import dotenv</font> and [**ChatOpenAI**](https://python.langchain.com/docs/integrations/chat/openai). Then you call <font color='blue'>dotenv.load_dotenv()</font> which reads and stores environment variables from <font color='olive'>.env</font>. By default, <font color='blue'>dotenv.load_dotenv()</font> assumes <font color='olive'>.env</font> is located in the current working directory, but you can pass the path to other directories if <font color='olive'>.env</font> is located elsewhere.

**You then instantiate a [ChatOpenAI](https://python.langchain.com/docs/integrations/chat/openai) model using GPT 3.5 Turbo as the base LLM, and you set `temperature` to 0**. OpenAI offers a diversity of models with varying price points, capabilities, and performances. GPT 3.5 turbo is a great model to start with because it performs well in many use cases and is cheaper than more recent models like GPT 4 and beyond.

<b><font color='darkred'>Note</font></b>: It’s a common misconception that setting `temperature=0` guarantees deterministic responses from GPT models. <b>While responses are closer to deterministic when `temperature=0`, there’s no guarantee that you’ll get the same response for identical requests</b>. Because of this, GPT models might output slightly different results than what you see in the examples throughout this tutorial.

To use `chat_model`, open the project directory, start a Python interpreter, and run the following code:

In [3]:
from langchain.schema.messages import HumanMessage, SystemMessage
from langchain_intro.chat_bot import chat_model

messages = [
    SystemMessage(
        content="""You're an assistant knowledgeable about healthcare. Only answer healthcare-related questions."""
    ),
    HumanMessage(content="What is Medicaid managed care?"),
]

In [4]:
chat_model.invoke(messages)

AIMessage(content='Medicaid managed care is a system in which states contract with managed care organizations (MCOs) to provide healthcare services to Medicaid beneficiaries. These MCOs are responsible for coordinating and delivering healthcare services to enrollees in exchange for a fixed monthly payment per enrollee. Medicaid managed care aims to improve access to care, enhance quality, and control costs for Medicaid beneficiaries.')

In this block, you import [**HumanMessage**](https://python.langchain.com/docs/modules/model_io/concepts#humanmessage) and [**SystemMessage**](https://python.langchain.com/docs/modules/model_io/concepts#systemmessage), as well as your chat model. You then define a list with a [**SystemMessage**](https://python.langchain.com/docs/modules/model_io/concepts#systemmessage) and a [**HumanMessage**](https://python.langchain.com/docs/modules/model_io/concepts#humanmessage) and run them through `chat_model` with <font color='blue'>chat_model.invoke()</font>. Under the hood, `chat_model` makes a request to an OpenAI endpoint serving gpt-3.5-turbo-0125, and the results are returned as an [**AIMessage**](https://python.langchain.com/docs/modules/model_io/concepts#aimessage).

As you can see, the chat model answered `What is Medicaid managed care?` provided in the [**HumanMessage**](https://api.python.langchain.com/en/latest/messages/langchain_core.messages.human.HumanMessage.html). You might be wondering what the chat model did with the [**SystemMessage**](https://api.python.langchain.com/en/latest/messages/langchain_core.messages.system.SystemMessage.html) in this context. Notice what happens when you ask the following question:

In [6]:
messages = [
    SystemMessage(
        content="""You're an assistant knowledgeable about healthcare. Only answer healthcare-related questions."""),
    HumanMessage(content="How do I change a tire?"),
]

In [7]:
chat_model.invoke(messages)

AIMessage(content="I'm here to help with healthcare-related questions. If you have any health-related inquiries, feel free to ask!")

<b><font size='3ptx'>As described earlier, the [SystemMessage](https://api.python.langchain.com/en/latest/messages/langchain_core.messages.system.SystemMessage.html) tells the model how to behave</font></b>. In this case, you told the model to only answer healthcare-related questions. This is why it refuses to tell you how to change your tire. <b>The ability to control how an LLM relates to the user through text instructions is powerful, and this is the foundation for creating customized chatbots through prompt engineering</b>.

While chat messages are a nice abstraction and are good for ensuring that you’re giving the LLM the right kind of message, <b>you can also pass raw strings into chat models</b>:

In [8]:
chat_model.invoke("What is blood pressure?")

AIMessage(content='Blood pressure is the force of blood pushing against the walls of the arteries as the heart pumps blood throughout the body. It is measured in millimeters of mercury (mmHg) and is typically recorded as two numbers: systolic pressure (the top number) and diastolic pressure (the bottom number). A normal blood pressure reading is typically around 120/80 mmHg. High blood pressure (hypertension) can increase the risk of heart disease, stroke, and other health problems, while low blood pressure (hypotension) can cause dizziness, fainting, and other symptoms.')

In this code block, you pass the string `What is blood pressure?` directly to <font color='blue'>chat_model.invoke()</font>. <b>If you want to control the LLM’s behavior without a [SystemMessage](https://api.python.langchain.com/en/latest/messages/langchain_core.messages.system.SystemMessage.html) here, you can include instructions in the string input</b>.

<b><font color='darkred'>Note:</font></b>
> In these examples, you used `.invoke()`, but LangChain has [other methods](https://python.langchain.com/docs/expression_language/interface) that interact with LLMs. For instance, `.stream()` returns the response one token at time, and `.batch()` accepts a list of messages that the LLM responds to in one call.
> Each method also has an analogous asynchronous method. For instance, you can run `.invoke()` asynchronously with `ainvoke()`.

<br/>

Next up, you’ll learn a modular way to guide your model’s response, as you did with the [**SystemMessage**](https://api.python.langchain.com/en/latest/messages/langchain_core.messages.system.SystemMessage.html), making it easier to customize your chatbot.

<a id='step1_2'></a>
### <b><font color='darkgreen'>Prompt Templates</font></b>
<b><font size='3ptx'>LangChain allows you to design modular prompts for your chatbot with [prompt templates](https://python.langchain.com/docs/modules/model_io/prompts/quick_start).</font></b>

Quoting LangChain’s documentation, <b>you can think of prompt templates as predefined recipes for generating prompts for language models</b>.

Suppose you want to build a chatbot that answers questions about patient experiences from their reviews. Here’s what a prompt template might look like for this:

In [14]:
from langchain.prompts import ChatPromptTemplate

review_template_str = """Your job is to use patient
reviews to answer questions about their experience at a hospital.
Use the following context to answer questions. Be as detailed
as possible, but don't make up any information that's not
from the context. If you don't know an answer, say you don't know.

{context}

{question}"""

review_template = ChatPromptTemplate.from_template(review_template_str)
context = "I had a great stay!"
question = "Did anyone have a positive experience?"

In [16]:
print(review_template.format(context=context, question=question))

Human: Your job is to use patient
reviews to answer questions about their experience at a hospital.
Use the following context to answer questions. Be as detailed
as possible, but don't make up any information that's not
from the context. If you don't know an answer, say you don't know.

I had a great stay!

Did anyone have a positive experience?


You first import [**ChatPromptTemplate**](https://api.python.langchain.com/en/latest/prompts/langchain_core.prompts.chat.ChatPromptTemplate.html) and define `review_template_str`, which contains the instructions that you’ll pass to the model, along with the variables `context` and `question` in [replacement fields](https://realpython.com/python-f-strings/#the-strformat-method) that LangChain delimits with curly braces (`{}`). You then create a [**ChatPromptTemplate**](https://api.python.langchain.com/en/latest/prompts/langchain_core.prompts.chat.ChatPromptTemplate.html) object from `review_template_str` using the class method `.from_template()`.

With `review_template` instantiated, you can pass `context` and `question` into the string template with <font color='blue'>review_template.format()</font>. The results may look like you’ve done nothing more than [standard Python string interpolation](https://realpython.com/python-f-strings/), but prompt templates have a lot of useful features that allow them to integrate with chat models.

Notice how your previous call to <font color='blue'>review_template.format()</font> generated a string with `Human` at the beginning. <b>This is because <font color='blue'>**ChatPromptTemplate**.from_template()</font> assumes the string template is a human message by default. To change this, you can create more detailed prompt templates for each chat message that you want the model to process</b>:

In [24]:
from langchain.prompts import (
        PromptTemplate,
        SystemMessagePromptTemplate,
        HumanMessagePromptTemplate,
        ChatPromptTemplate,
)

review_system_template_str = """Your job is to use patient
reviews to answer questions about their experience at a
hospital. Use the following context to answer questions.
Be as detailed as possible, but don't make up any information
that's not from the context. If you don't know an answer, say
you don't know.

{context}
"""

review_system_prompt = SystemMessagePromptTemplate(
    prompt=PromptTemplate(
        input_variables=["context"], template=review_system_template_str
    )
)

review_human_prompt = HumanMessagePromptTemplate(
    prompt=PromptTemplate(
        input_variables=["question"], template="{question}"
)
)

messages = [review_system_prompt, review_human_prompt]
review_prompt_template = ChatPromptTemplate(
input_variables=["context", "question"],
    messages=messages,
)
context = "I had a great stay!"
question = "Did anyone have a positive experience?"

for message in review_prompt_template.format_messages(context=context, question=question):
    print(f'>>> {message.__class__}:\n{message.content}\n')

>>> <class 'langchain_core.messages.system.SystemMessage'>:
Your job is to use patient
reviews to answer questions about their experience at a
hospital. Use the following context to answer questions.
Be as detailed as possible, but don't make up any information
that's not from the context. If you don't know an answer, say
you don't know.

I had a great stay!


>>> <class 'langchain_core.messages.human.HumanMessage'>:
Did anyone have a positive experience?



To see how to combine chat models and prompt templates, you’ll build a chain with the [**LangChain Expression Language**](https://python.langchain.com/docs/expression_language/) (LCEL). <b>This helps you unlock LangChain’s core functionality of building modular customized interfaces over chat models</b>.

<a id='step1_3'></a>
### <b><font color='darkgreen'>Chains and LangChain Expression Language (LCEL)</font></b>
<b><font size='3ptx'>The glue that connects chat models, prompts, and other objects in LangChain is the [chain](https://python.langchain.com/docs/modules/chains).</font></b>

<b>A chain is nothing more than a sequence of calls between objects in LangChain</b>. The recommended way to build chains is to use the [**LangChain Expression Language**](https://python.langchain.com/docs/expression_language/) (LCEL). To see how this works, take a look at how you’d create a chain with a chat model and prompt template:

```python
import dotenv
from langchain_openai import ChatOpenAI
from langchain.prompts import (
    PromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    ChatPromptTemplate,
)

dotenv.load_dotenv()

review_template_str = """Your job is to use patient
reviews to answer questions about their experience at
a hospital. Use the following context to answer questions.
Be as detailed as possible, but don't make up any information
that's not from the context. If you don't know an answer, say
you don't know.

{context}
"""

review_system_prompt = SystemMessagePromptTemplate(
    prompt=PromptTemplate(
        input_variables=["context"],
        template=review_template_str,
    )
)

review_human_prompt = HumanMessagePromptTemplate(
    prompt=PromptTemplate(
        input_variables=["question"],
        template="{question}",
    )
)
messages = [review_system_prompt, review_human_prompt]

review_prompt_template = ChatPromptTemplate(
    input_variables=["context", "question"],
    messages=messages,
)

chat_model = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)

review_chain = review_prompt_template | chat_model
```

This creates an object, `review_chain`, that can pass questions through `review_prompt_template` and `chat_model` in a single function call. In essence, this abstracts away all of the internal details of `review_chain`, allowing you to interact with the chain as if it were a chat model.

After saving the updated <font color='olive'>chatbot.py</font>, start a new REPL session in your base project folder. Here’s how you can use `review_chain`:

In [2]:
from langchain_intro.chat_bot import review_chain

context = "I had a great stay!"
question = "Did anyone have a positive experience?"

In [3]:
review_chain.invoke({"context": context, "question": question})

AIMessage(content='Yes, the patient who left the review had a positive experience during their stay at the hospital.')

In this block, you import `review_chain` and define context and question as before. You then pass a dictionary with the keys `context` and `question` into <font color='blue'>review_chan.invoke()</font>. This passes `context` and `question` through the prompt template and chat model to generate an answer.

<b><font color='darkred'>Note:</font></b> When calling chains, you can use all of the same methods that a chat model supports.

In general, the LCEL allows you to create arbitrary-length chains with the pipe symbol (`|`). For instance, if you wanted to format the model’s response, then you could add an output parser to the chain:

```python
import dotenv
from langchain_openai import ChatOpenAI
from langchain.prompts import (
    PromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    ChatPromptTemplate,
)
from langchain_core.output_parsers import StrOutputParser

# ...

output_parser = StrOutputParser()

review_chain = review_prompt_template | chat_model
review_chain_with_stdout = review_chain | output_parser
```

Here, you add a [**StrOutputParser**](https://api.python.langchain.com/en/latest/output_parsers/langchain_core.output_parsers.string.StrOutputParser.html) instance to `review_chain`, which will make the model’s response more readable. Start a new REPL session and give it a try:

In [4]:
from langchain_intro.chat_bot import review_chain_with_stdout

context = "I had a great stay!"
question = "Did anyone have a positive experience?"

In [5]:
review_chain_with_stdout.invoke({"context": context, "question": question})

'Yes, the patient who left the review had a great stay, indicating a positive experience at the hospital.'

This block is the same as before, except now you can see that `review_chain_with_stdout` returns a nicely-formatted string rather than an [**AIMessage**](https://api.python.langchain.com/en/latest/messages/langchain_core.messages.ai.AIMessage.html).

<b>The power of chains is in the creativity and flexibility they afford you. You can chain together complex pipelines to create your chatbot, and you end up with an object that executes your pipeline in a single method call</b>. Next up, you’ll layer another object into `review_chain_with_stdout` to retrieve documents from a vector database.

<a id='step1_4'></a>
### <b><font color='darkgreen'>Retrieval Objects</font></b>
<b><font size='3ptx'>The goal of `review_chain_with_stdout` is to answer questions about patient experiences in the hospital from their reviews.</font></b>

<b>So far, you’ve manually passed reviews in as context for the question. While this can work for a small number of reviews, it doesn’t scale well</b>. Moreover, even if you can fit all reviews into the model’s context window, there’s no guarantee it will use the correct reviews when answering a question.

<b>To overcome this, you need a [retriever](https://python.langchain.com/docs/modules/data_connection/). The process of retrieving relevant documents and passing them to a language model to answer questions is known as [retrieval-augmented generation](https://en.wikipedia.org/wiki/Prompt_engineering#Retrieval-augmented_generation)</b> (RAG).

For this example, you’ll store all the reviews in a [**vector database**](https://en.wikipedia.org/wiki/Vector_database) called [**ChromaDB**](https://www.trychroma.com/). If you’re unfamiliar with this database tool and topics, then check out [Embeddings and Vector Databases with ChromaDB](https://realpython.com/chromadb-vector-database/) before continuing.

You can install [**ChromaDB**](https://www.trychroma.com/) with the following command:

In [7]:
#!python -m pip install chromadb==0.4.22

With this installed, you can use the following code to create a ChromaDB vector database with patient reviews:
* <font color='olive'>`langchain_intro/create_retriever.py`</font>

```python
import dotenv
from langchain.document_loaders.csv_loader import CSVLoader
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

REVIEWS_CSV_PATH = "data/reviews.csv"
REVIEWS_CHROMA_PATH = "chroma_data"

dotenv.load_dotenv()

# lines 11 and 12
loader = CSVLoader(file_path=REVIEWS_CSV_PATH, source_column="review")
reviews = loader.load()

# lines 14 to 16
reviews_vector_db = Chroma.from_documents(
    reviews, OpenAIEmbeddings(), persist_directory=REVIEWS_CHROMA_PATH
)
```

In lines 11 and 12, you load the reviews using LangChain’s [**CSVLoader**](https://api.python.langchain.com/en/latest/document_loaders/langchain_community.document_loaders.csv_loader.CSVLoader.html). In lines 14 to 16, you create a ChromaDB instance from reviews using the default [**OpenAI embedding model**](https://platform.openai.com/docs/guides/embeddings), and you store the review embeddings at `REVIEWS_CHROMA_PATH`.

<b><font color='darkred'>Note:</font></b>
> In practice, if you’re embedding a large document, you should use a [**text splitter**](https://python.langchain.com/docs/modules/data_connection/document_transformers/). <b>Text splitters break the document into smaller chunks before running them through an embedding model</b>. This is important because embedding models have a fixed-size context window, and as the size of the text grows, an embedding’s ability to accurately represent the text decreases.
> <br/><br/>
> For this example, you can embed each review individually because they’re relatively small.

<br/>

Next, open a terminal and run the following command from the project directory:

```shell
(venv) $ python langchain_intro/create_retriever.py
```

It should only take a minute or so to run, and afterwards you can start <b>performing semantic search over the review embeddings</b>:

In [8]:
import dotenv
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

REVIEWS_CHROMA_PATH = "chroma_data/"

dotenv.load_dotenv()

reviews_vector_db = Chroma(
    persist_directory=REVIEWS_CHROMA_PATH,
    embedding_function=OpenAIEmbeddings(),
)

In [9]:
question = """Has anyone complained about communication with the hospital staff?"""
relevant_docs = reviews_vector_db.similarity_search(question, k=3)

In [11]:
print(relevant_docs[0].page_content)

review_id: 73
visit_id: 7696
review: I had a frustrating experience at the hospital. The communication between the medical staff and me was unclear, leading to misunderstandings about my treatment plan. Improvement is needed in this area.
physician_name: Maria Thompson
hospital_name: Little-Spencer
patient_name: Terri Smith


In [12]:
print(relevant_docs[1].page_content)

review_id: 521
visit_id: 631
review: I had a challenging time at the hospital. The medical care was adequate, but the lack of communication between the staff and me left me feeling frustrated and confused about my treatment plan.
physician_name: Samantha Mendez
hospital_name: Richardson-Powell
patient_name: Kurt Gordon


In [13]:
print(relevant_docs[2].page_content)

review_id: 785
visit_id: 2593
review: My stay at the hospital was challenging. The medical care was adequate, but the lack of communication from the staff created some frustration.
physician_name: Brittany Harris
hospital_name: Jones, Taylor and Garcia
patient_name: Ryan Jacobs


You import the dependencies needed to call ChromaDB and specify the path to the stored ChromaDB data in `REVIEWS_CHROMA_PATH`. You then load environment variables using <font color='blue'>dotenv.load_dotenv()</font> and create a new Chroma instance pointing to your vector database. <b>Notice how you have to specify an embedding function again when connecting to your vector database. Be sure this is the same embedding function that you used to create the embeddings</b>.

Next, you define a question and call <font color='blue'>.similarity_search()</font> on `reviews_vector_db`, passing in `question` and `k=3`. This creates an embedding for the question and searches the vector database for the three most similar review embeddings to question embedding. In this case, you see three reviews where patients complained about communication, which is exactly what you asked for!

<b>The last thing to do is add your reviews retriever to `review_chain_with_stdout` so that relevant reviews are passed to the prompt as context</b>. Here’s how you do that:

- `langchain_intro/chatbot.py`

```python
import dotenv
from langchain_openai import ChatOpenAI
from langchain.prompts import (
    PromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    ChatPromptTemplate,
)
from langchain_core.output_parsers import StrOutputParser
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain.schema.runnable import RunnablePassthrough

REVIEWS_CHROMA_PATH = "chroma_data/"

# ...

reviews_vector_db = Chroma(
    persist_directory=REVIEWS_CHROMA_PATH,
    embedding_function=OpenAIEmbeddings()
)

reviews_retriever  = reviews_vector_db.as_retriever(k=10)

review_chain_with_rag = (
    {"context": reviews_retriever, "question": RunnablePassthrough()}  # New here
    | review_prompt_template
    | chat_model
    | StrOutputParser()
)
```

As before, you import ChromaDB’s dependencies, specify the path to your ChromaDB data, and instantiate a new Chroma object. You then create `reviews_retriever` by calling <font color='blue'>.as_retriever()</font> on `reviews_vector_db` to create a retriever object that you’ll add to `review_chain_with_rag`. Because you specified `k=10`, the retriever will fetch the ten reviews most similar to the user’s question.

You then add a dictionary with c`ontext` and `question` keys to the front of `review_chain_with_rag`. Instead of passing `context` in manually, `review_chain_with_rag` will pass your `question` to the retriever to pull relevant reviews. <b>Assigning `question` to a [**RunnablePassthrough**](https://python.langchain.com/docs/expression_language/primitives/passthrough/) object ensures the `question` gets passed unchanged to the next step in the chain</b>.

You now have a fully functioning chain that can answer questions about patient experiences from their reviews. Start a new REPL session and try it out:

In [3]:
from langchain_intro.chat_bot import review_chain_with_rag

question = """Has anyone complained about communication with the hospital staff?"""

In [4]:
review_chain_with_rag.invoke(question)

'Yes, several patients have complained about communication with the hospital staff. For example, Terri Smith mentioned that the communication between the medical staff and her was unclear, leading to misunderstandings about her treatment plan. Kurt Gordon also expressed frustration and confusion about his treatment plan due to a lack of communication between the staff and himself. Ryan Jacobs and Shannon Williams also mentioned challenges and frustration caused by a lack of communication from the hospital staff.'

As you can see, you only call <font color='blue'>review_chain_with_rag.invoke(question)</font> to get retrieval-augmented answers about patient experiences from their reviews. You’ll improve upon this chain later by storing review embeddings, along with other metadata, in Neo4j.

Now that you understand chat models, prompts, chains, and retrieval, you’re ready to dive into the last LangChain concept—agents.

<a id='step1_5'></a>
### <b><font color='darkgreen'>Agents</font></b>
<b><font size='3ptx'>So far, you’ve created a chain to answer questions using patient reviews. What if you want your chatbot to also answer questions about other hospital data, such as hospital wait times? </font></b>

Ideally, your chatbot can seamlessly switch between answering patient review and wait time questions depending on the user’s query. To accomplish this, you’ll need the following components:
1. The patient review chain you already created.
2. A function that can look up wait times at a hospital.
3. A way for an LLM to know when it should answer questions about patient experiences or look up wait times.

To accomplish the third capability, you need an [**agent**](https://python.langchain.com/docs/modules/agents/).

<b>An agent is a language model that decides on a sequence of actions to execute</b>. Unlike chains where the sequence of actions is hard-coded, agents use a language model to determine which actions to take and in which order.

Before building the agent, <b>create the following function to generate fake wait times for a hospital</b>:

In [1]:
import random
import time

def get_current_wait_time(hospital: str) -> int | str:
    """Dummy function to generate fake wait times"""

    if hospital not in ["A", "B", "C", "D"]:
        return f"Hospital {hospital} does not exist"

    # Simulate API call delay
    time.sleep(1)

    return random.randint(0, 10000)

<b>In `get_current_wait_time()`, you pass in a hospital name, check if it’s valid, and then generate a random number to simulate a wait time</b>. In reality, this would be some sort of database query or API call, but this will serve the same purpose for this demonstration.

You can now create an agent that decides between `get_current_wait_time()` and `review_chain_with_rag.invoke()` depending on the question:

* <font color='olive'>`langchain_intro/chatbot.py`</font>

```python
import dotenv
from langchain_openai import ChatOpenAI
from langchain.prompts import (
    PromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    ChatPromptTemplate,
)
from langchain_core.output_parsers import StrOutputParser
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain.schema.runnable import RunnablePassthrough
from langchain.agents import (
    create_openai_functions_agent,
    Tool,
    AgentExecutor,
)
from langchain import hub
from langchain_intro.tools import get_current_wait_time

# ...

tools = [
    Tool(
        name="Reviews",
        func=review_chain.invoke,
        description="""Useful when you need to answer questions
        about patient reviews or experiences at the hospital.
        Not useful for answering questions about specific visit
        details such as payer, billing, treatment, diagnosis,
        chief complaint, hospital, or physician information.
        Pass the entire question as input to the tool. For instance,
        if the question is "What do patients think about the triage system?",
        the input should be "What do patients think about the triage system?"
        """,
    ),
    Tool(
        name="Waits",
        func=get_current_wait_time,
        description="""Use when asked about current wait times
        at a specific hospital. This tool can only get the current
        wait time at a hospital and does not have any information about
        aggregate or historical wait times. This tool returns wait times in
        minutes. Do not pass the word "hospital" as input,
        only the hospital name itself. For instance, if the question is
        "What is the wait time at hospital A?", the input should be "A".
        """,
    ),
]

hospital_agent_prompt = hub.pull("hwchase17/openai-functions-agent")

agent_chat_model = ChatOpenAI(
    model="gpt-3.5-turbo-1106",
    temperature=0,
)

hospital_agent = create_openai_functions_agent(
    llm=agent_chat_model,
    prompt=hospital_agent_prompt,
    tools=tools,
)

hospital_agent_executor = AgentExecutor(
    agent=hospital_agent,
    tools=tools,
    return_intermediate_steps=True,
    verbose=True,
)
```

In this block, you import a few additional dependencies that you’ll need to create the agent. You then define a list of [**Tool**](https://python.langchain.com/docs/modules/agents/tools/) objects. A [**Tool**](https://python.langchain.com/docs/modules/agents/tools/) is an interface that an agent uses to interact with a function. For instance, the first tool is named `Reviews` and it calls <font color='blue'>review_chain.invoke()</font> if the question meets the criteria of `description`.

<b>Notice how `description` gives the agent instructions as to when it should call the tool</b>. This is where good prompt engineering skills are paramount to ensuring the LLM calls the correct tool with the correct inputs.

The second [**Tool**](https://python.langchain.com/docs/modules/agents/tools/) in tools is named `Waits`, and it calls <font color='blue'>get_current_wait_time()</font>. Again, <b>the agent has to know when to use the `Waits` tool and what inputs to pass into it depending on the `description`</b>.

Next, you initialize a ChatOpenAI object using **gpt-3.5-turbo-1106** as your language model. You then create an OpenAI functions agent with <font color='blue'>create_openai_functions_agent()</font>. This creates an agent designed to pass inputs to functions. It does this by returning valid JSON objects that store function inputs and their corresponding value.

To create the agent run time, you pass the agent and tools into [**AgentExecutor**](https://python.langchain.com/docs/modules/agents/concepts#agentexecutor). Setting `return_intermediate_steps` and `verbose` to True will allow you to see the agent’s thought process and the tools it calls.

Start a new REPL session to give your new agent a spin:

In [5]:
from langchain_intro.chat_bot import hospital_agent_executor

In [6]:
hospital_agent_executor.invoke(
    {"input": "What is the current wait time at hospital C?"})



> Entering new AgentExecutor chain...

Invoking: `Waits` with `C`


1215The current wait time at hospital C is 12-15 minutes.

> Finished chain.


{'input': 'What is the current wait time at hospital C?',
 'output': 'The current wait time at hospital C is 12-15 minutes.',
 'intermediate_steps': [(AgentActionMessageLog(tool='Waits', tool_input='C', log='\nInvoking: `Waits` with `C`\n\n\n', message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"__arg1":"C"}', 'name': 'Waits'}})]),
   1215)]}

You first import the agent and then call <font color='blue'>hospital_agent_executor.invoke()</font> with a question about a wait time. As indicated in the output, the agent knows that you’re asking about a wait time, and it passes `C` as input to the `Waits` tool. The `Waits` tool then calls <font color='blue'>get_current_wait_time(hospital="C")</font> and returns the corresponding wait time to the agent. The agent then uses this wait time to generate its final output.

In [7]:
hospital_agent_executor.invoke(
    {"input": "What have patients said about their comfort at the hospital?"})



> Entering new AgentExecutor chain...

Invoking: `Reviews` with `What have patients said about their comfort at the hospital?`


Patients have mentioned both positive and negative aspects of their comfort at the hospital. One patient appreciated the hospital's dedication to patient comfort, mentioning well-designed private rooms and comfortable furnishings that made their recovery more bearable and contributed to an overall positive experience. However, other patients have complained about uncomfortable beds affecting their ability to get a good night's sleep during their stay, which impacted their overall comfort despite other positive aspects like knowledgeable doctors and a clean environment.Patients have mentioned both positive and negative aspects of their comfort at the hospital. One patient appreciated the hospital's dedication to patient comfort, mentioning well-designed private rooms and comfortable furnishings that made their recovery more bearable and contributed to an ove

{'input': 'What have patients said about their comfort at the hospital?',
 'output': "Patients have mentioned both positive and negative aspects of their comfort at the hospital. One patient appreciated the hospital's dedication to patient comfort, mentioning well-designed private rooms and comfortable furnishings that made their recovery more bearable and contributed to an overall positive experience. However, other patients have complained about uncomfortable beds affecting their ability to get a good night's sleep during their stay, which impacted their overall comfort despite other positive aspects like knowledgeable doctors and a clean environment.",
 'intermediate_steps': [(AgentActionMessageLog(tool='Reviews', tool_input='What have patients said about their comfort at the hospital?', log='\nInvoking: `Reviews` with `What have patients said about their comfort at the hospital?`\n\n\n', message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"__arg1"

A similar process happens when you ask the agent about patient experience reviews, except this time the agent knows to call the `Reviews` tool with `What have patients said about their comfort at the hospital?` as input. The `Reviews` tool runs <font color='blue'>review_chain.invoke()</font> using your full question as input, and the agent uses the response to generate its output.

This is a profound capability. Agents give language models the ability to perform just about any task that you can write code for. Imagine all of the amazing, and potentially dangerous, chatbots you could build with agents.

You now have all of the prerequisite LangChain knowledge needed to build a custom chatbot. Next up, you’ll put on your AI engineer hat and learn about the business requirements and data needed to build your hospital system chatbot.

All of the code you’ve written so far was intended to teach you the fundamentals of LangChain, and it won’t be included in your final chatbot. Feel free to start with an empty directory in Step 2, where you’ll begin building your chatbot.

<a id='step2'></a>
## <b><font color='darkblue'>Step 2: Understand the Business Requirements and Data</font></b> ([back](#agenda))
* <b><font size='3ptx'><a href='#step2_1'>Understand the Problem and Requirements</a></font></b>
* <b><font size='3ptx'><a href='#step2_2'>Explore the Available Data</a></font></b>
* <b><font size='3ptx'><a href='#step2_3'>Wait Times</a></font></b>
* <b><font size='3ptx'><a href='#step2_4'>Design the Chatbot</a></font></b>

<b><font size='3ptx'>Before you start working on any AI project, you need to understand the problem that you want to solve and make a plan for how you’re going to solve it.</font></b>

This involves clearly defining the problem, gathering requirements, understanding the data and technology available to you, and setting clear expectations with stakeholders. For this project, you’ll start by defining the problem and gathering business requirements for your chatbot.

<a id='step2_1'></a>
### <b><font color='darkgreen'>Understand the Problem and Requirements</font></b>
<b><font size='3ptx'>Imagine you’re an AI engineer working for a large hospital system in the US. </font></b>

<b>Your stakeholders would like more visibility into the ever-changing data they collect</b>. They want answers to ad-hoc questions about patients, visits, physicians, hospitals, and insurance payers without having to understand a query language like SQL, request a report from an analyst, or wait for someone to build a dashboard.

<b>To accomplish this, your stakeholders want an internal chatbot tool, similar to ChatGPT, that can answer questions about your company’s data</b>. After meeting to gather requirements, you’re provided with a list of the kinds of questions your chatbot should answer:
* What is the current wait time at XYZ hospital?
* Which hospital currently has the shortest wait time?
* At which hospitals are patients complaining about billing and insurance issues?
* Have any patients complained about the hospital being unclean?
* What have patients said about how doctors and nurses communicate with them?
* What are patients saying about the nursing staff at XYZ hospital?
* What was the total billing amount charged to Cigna payers in 2023?
* How many patients has Dr. John Doe treated?
* How many visits are open and what is their average duration in days?
* Which physician has the lowest average visit duration in days?
* How much was billed for patient 789’s stay?
* Which hospital worked with the most Cigna patients in 2023?
* What’s the average billing amount for emergency visits by hospital?
* Which state had the largest percent increase inedicaid visits from 2022 to 2023?

<br/>

You can answer questions like What was the total billing amount charged to Cigna payers in 2023? with aggregate statistics using a query language like SQL. Crucially, these questions have a single objective answer. <b>You could run pre-defined queries to answer these, but any time a stakeholder has a new or slightly nuanced question, you have to write a new query. To avoid this, your chatbot should dynamically generate accurate queries</b>.

Questions like `Have any patients complained about the hospital being unclean?` or `What have patients said about how doctors and nurses communicate with them?` are more subjective and might have many acceptable answers. Your chatbot will need to read through documents, such as patient reviews, to answer these kinds of questions.

Ultimately, your stakeholders want a single chat interface that can seamlessly answer both subjective and objective questions. This means, when presented with a question, your chatbot needs to know what type of question is being asked and which data source to pull from.

For instance, if asked `How much was billed for patient 789’s stay?`, your chatbot should know it needs to query a database to find the answer. If asked `What have patients said about how doctors and nurses communicate with them?`, your chatbot should know it needs to read and summarize patient reviews.

<b>Next up, you’ll explore the data your hospital system records, which is arguably the most important prerequisite to building your chatbot</b>.

<a id='step2_2'></a>
### <b><font color='darkgreen'>Explore the Available Data</font></b> ([back](#step2))
<b><font size='3ptx'>Before building your chatbot, you need a thorough understanding of the data it will use to respond to user queries.</font></b>

This will help you determine what’s feasible and how you want to structure the data so that your chatbot can easily access it. All of the data you’ll use in this article was synthetically generated, and much of it was derived from a popular [**health care dataset on Kaggle**](https://www.kaggle.com/datasets/prasad22/healthcare-dataset).

In practice, the following datasets would likely be stored as tables in a SQL database, but <b>you’ll work with CSV files to keep the focus on building the chatbot. This section will give you a detailed description of each CSV file</b>.

You’ll need to place all CSV files that are part of this project in your <font color='olive'>data/</font> folder before continuing with the tutorial. <b>Make sure that you downloaded them from the materials and placed them in your <font color='olive'>data/</font> folder</b> ([Github link](https://github.com/realpython/materials/tree/master/langchain-rag-app/source_code_final)):

#### <b>`hospitals.csv`</font>
The <font color='olive'>hospitals.csv</font> file records information on each hospital that your company manages. There 30 hospitals and three [fields](https://en.wikipedia.org/wiki/Field_(computer_science)) in this file:
* **hospital_id**: An integer that uniquely identifies a hospital.
* **hospital_name**: The hospital’s name.
* **hospital_state**: The state the hospital is located in.

If you’re familiar with traditional SQL databases and the [**star schema**](https://en.wikipedia.org/wiki/Star_schema), you can think of <font color='olive'>hospitals.csv</font> as a [**dimension table**](https://en.wikipedia.org/wiki/Star_schema#Dimension_tables). Dimension tables are relatively short and contain descriptive information or attributes that provide context to the data in [**fact tables**](https://en.wikipedia.org/wiki/Star_schema#Fact_tables). Fact tables record events about the entities stored in dimension tables, and they tend to be longer tables.

In this case, <font color='olive'>hospitals.csv</font> records information specific to hospitals, but you can join it to fact tables to answer questions about which patients, physicians, and payers are related to the hospital. This will be more clear when you explore <font color='olive'>visits.csv</font>.

If you’re curious, you can inspect the first few rows of <font color='olive'>hospitals.csv</font> using a dataframe library like [**Polars**](https://docs.pola.rs/user-guide/getting-started/). Make sure [**Polars**](https://docs.pola.rs/user-guide/getting-started/) is installed in your virtual environment, and run the following code:

In [2]:
import polars as pl

HOSPITAL_DATA_PATH = "data/hospitals.csv"
data_hospitals = pl.read_csv(HOSPITAL_DATA_PATH)
print(data_hospitals.shape)

(30, 3)


In [8]:
data_hospitals.head()

hospital_id,hospital_name,hospital_state
i64,str,str
0,"""Wallace-Hamilt…","""CO"""
1,"""Burke, Griffin…","""NC"""
2,"""Walton LLC""","""FL"""
3,"""Garcia Ltd""","""NC"""
4,"""Jones, Brown a…","""NC"""


This shows you, for example, that `Walton, LLC` hospital has an ID of `2` and is located in the state of Florida, `FL`.

#### <b>`physicians.csv`</b>
The <font color='olive'>`physicians.csv`</font> file contains data about the physicians that work for your hospital system. This dataset has the following fields:
* **physician_id**: An integer that uniquely identifies each physician.
* **physician_name**: The physician’s name.
* **physician_dob**: The physician’s date of birth.
* **physician_grad_year**: The year the physician graduated medical school.
* **medical_school**: Where the physician attended medical school.
* **salary**: The physician’s salary.

This data can again be thought of as a dimension table, and you can inspect the first few rows using Polars:

In [10]:
PHYSICIAN_DATA_PATH = "data/physicians.csv"
data_physician = pl.read_csv(PHYSICIAN_DATA_PATH)
print(data_physician.shape)

(500, 6)


In [11]:
data_physician.head()

physician_name,physician_id,physician_dob,physician_grad_year,medical_school,salary
str,i64,str,str,str,f64
"""Joseph Johnson…",0,"""1970-02-22""","""2000-02-22""","""Johns Hopkins …",309534.155076
"""Jason Williams…",1,"""1982-12-22""","""2012-12-22""","""Mayo Clinic Al…",281114.503559
"""Jesse Gordon""",2,"""1959-06-03""","""1989-06-03""","""David Geffen S…",305845.584636
"""Heather Smith""",3,"""1965-06-15""","""1995-06-15""","""NYU Grossman M…",295239.766689
"""Kayla Hunter D…",4,"""1978-10-19""","""2008-10-19""","""David Geffen S…",298751.355201


For instance, `Heather Smith` has a physician ID of `3`, was born on `June 15, 1965`, graduated medical school on `June 15, 1995`, attended `NYU Grossman Medical School`, and her salary is about `$295,239`.

#### <b>`payers.csv`</b>
The next file, <font color='olive'>`payers.csv`</font>, records information about the insurance companies that your hospitals bills for patient visits. Similar to hospitals.csv, it’s a small file with a couple fields:
* **payer_id**: An integer that uniquely identifies each payer.
* **payer_name**: The payer’s company name.

The only five payers in the data are **Medicaid**, **UnitedHealthcare**, **Aetna**, **Cigna**, and **Blue Cross**. Your stakeholders are very interested in payer activity, so payers.csv will be helpful once it’s connected to patients, hospitals, and physicians.

#### <b>`reviews.csv`</b>
The <font color='olive'>reviews.csv</font> file contains patient reviews about their experience at the hospital. It has these fields:
* **review_id**: An integer that uniquely identifies a review.
* **visit_id**: An integer that identifies the patient’s visit that the review was about.
* **review**: This is the free form text review left by the patient.
* **physician_name**: The name of the physician who treated the patient.
* **hospital_name**: The hospital where the patient stayed.
* **patient_name**: The patient’s name.

This dataset is the first one you’ve seen that contains the free text **review** field, and your chatbot should use this to answer questions about review details and patient experiences. Here’s what <font color='olive'>reviews.csv</font> looks like:

In [3]:
REVIEWS_DATA_PATH = "data/reviews.csv"
data_reviews = pl.read_csv(REVIEWS_DATA_PATH)
print(data_reviews.shape)

(1005, 6)


In [4]:
data_reviews.head()

review_id,visit_id,review,physician_name,hospital_name,patient_name
i64,i64,str,str,str,str
0,6997,"""The medical st…","""Laura Brown""","""Wallace-Hamilt…","""Christy Johnso…"
9,8138,"""The hospital's…","""Steven Watson""","""Wallace-Hamilt…","""Anna Frazier"""
11,680,"""The hospital's…","""Chase Mcpherso…","""Wallace-Hamilt…","""Abigail Mitche…"
892,9846,"""I had a positi…","""Jason Martinez…","""Wallace-Hamilt…","""Kimberly Rivas…"
822,7397,"""The medical te…","""Chelsey Davis""","""Wallace-Hamilt…","""Catherine Yang…"


There are 1005 reviews in this dataset, and you can see how each review relates to a visit.

#### <b>`visits.csv`</b>
The last file, <font color='olive'>visits.csv</font>, records details about every hospital visit your company has serviced. Continuing with the star schema analogy, you can think of <font color='olive'>visits.csv</font> as a [**fact table**](https://en.wikipedia.org/wiki/Star_schema#Fact_tables) that connects hospitals, physicians, patients, and payers. Here are the fields:
* **visit_id**: The unique identifier of a hospital visit.
* **patient_id**: The ID of the patient associated with the visit.
* **date_of_admission**: The date the patient was admitted to the hospital.
* **room_number**: The patient’s room number.
* **admission_type**: One of ‘Elective’, ‘Emergency’, or ‘Urgent’.
* **chief_complaint**: A string describing the patient’s primary reason for being at the hospital.
* **primary_diagnosis**: A string describing the primary diagnosis made by the physician.
* **treatment_description**: A text summary of the treatment given by the physician.
* **test_results**: One of ‘Inconclusive’, ‘Normal’, or ‘Abnormal’.
* **discharge_date**: The date the patient was discharged from the hospital
* **physician_id**: The ID of the physician that treated the patient.
* **hospital_id**: The ID of the hospital the patient stayed at.
* **payer_id**: The ID of the insurance payer used by the patient.
* **billing_amount**: The amount of money billed to the payer for the visit.
* **visit_status**: One of ‘OPEN’ or ‘DISCHARGED’.

<b>This dataset gives you everything you need to answer questions about the relationship between each hospital entity</b>. For example, if you know a physician ID, you can use <font color='olive'>visits.csv</font> to figure out which patients, payers, and hospitals the physician is associated with. Take a look at what <font color='olive'>visits.csv</font> looks like in Polars:

In [6]:
VISITS_DATA_PATH = "data/visits.csv"
data_visits = pl.read_csv(VISITS_DATA_PATH)
data_visits.shape

(9998, 15)

In [7]:
data_visits.head()

patient_id,date_of_admission,billing_amount,room_number,admission_type,discharge_date,test_results,visit_id,physician_id,payer_id,hospital_id,chief_complaint,treatment_description,primary_diagnosis,visit_status
i64,str,f64,i64,str,str,str,i64,i64,i64,i64,str,str,str,str
0,"""2022-11-17""",37490.983364,146,"""Elective""","""2022-12-01""","""Inconclusive""",0,102,1,0,null,null,null,"""DISCHARGED"""
1,"""2023-06-01""",47304.064845,404,"""Emergency""",null,"""Normal""",1,435,4,5,null,null,null,"""OPEN"""
2,"""2019-01-09""",36874.896997,292,"""Emergency""","""2019-02-08""","""Normal""",2,348,2,6,null,null,null,"""DISCHARGED"""
3,"""2020-05-02""",23303.322092,480,"""Urgent""","""2020-05-03""","""Abnormal""",3,270,4,15,null,null,null,"""DISCHARGED"""
4,"""2021-07-09""",18086.344184,477,"""Urgent""","""2021-08-02""","""Normal""",4,106,2,29,"""Persistent cou…","""Prescribed a c…","""J45.909 - Unsp…","""DISCHARGED"""


You can see there are 9998 visits recorded along with the 15 fields described above. Notice that `chief_complaint`, `treatment_description`, and `primary_diagnosis` might be missing for a visit. You’ll have to keep this in mind as your stakeholders might not be aware that many visits are missing critical data—this may be a valuable insight in itself! Lastly, notice that when a visit is still open, the `discharged_date` will be missing.

You now have an understanding of the data you’ll use to build the chatbot your stakeholders want. To recap, the files are broken out to simulate what a traditional SQL database might look like. Every hospital, patient, physician, review, and payer are connected through <font color='olive'>visits.csv</font>.

<a id='step2_3'></a>
### <b><font color='darkgreen'>Wait Times</font></b> ([back](#step2))
You might have noticed there’s no data to answer questions like `What is the current wait time at XYZ hospital?` or Which hospital currently has the shortest wait time?. Unfortunately, the hospital system doesn’t record historical wait times. Your chatbot will have to call an API to get current wait time information. You’ll see how this works later.

<b>With an understanding of the business requirements, available data, and LangChain functionalities, you can create a design for your chatbot.</b>

<a id='step2_4'></a>
### <b><font color='darkgreen'>Design the Chatbot</font></b> ([back](#step2))
<b><font size='3ptx'>Now that you know the business requirements, data, and LangChain prerequisites, you’re ready to design your chatbot. </font></b>

<b>A good design gives you and others a conceptual understanding of the components needed to build your chatbot.</b> Your design should clearly illustrate how data flows through your chatbot, and it should serve as a helpful reference during development.

<b>Your design should clearly illustrate how data flows through your chatbot, and it should serve as a helpful reference during development.</b> Your chatbot will use multiple tools to answer diverse questions about your hospital system. Here’s a flowchart illustrating how you’ll accomplish this:
![flow chart](https://files.realpython.com/media/Screenshot_2024-01-15_at_8.08.18_PM.fe16f8a318cc.png)
<center><i>Architecture and data flow for the hospital system chatbot</i></center>

<br/>

This flowchart illustrates how data moves through your chatbot, starting from the user’s input query all the way to the final response. Here’s a summary of each component:
* **LangChain Agent**: The LangChain agent is the brain of your chatbot. Given a user query, the agent decides which tool to call and what to give the tool as input. The agent then observes the tool’s output and decides what to return to the user—this is the agent’s response.
* **Neo4j AuraDB**: You’ll store both structured hospital system data and patient reviews in a Neo4j AuraDB graph database. You’ll learn all about this in the next section.
* **LangChain Neo4j Cypher Chain**: This chain tries to convert the user query into Cypher, Neo4j’s query language, and execute the Cypher query in Neo4j. The chain then answers the user query using the Cypher query results. The chain’s response is fed back to the LangChain agent and sent to the user.
* **LangChain Neo4j Reviews Vector Chain**: This is very similar to the chain you built in [**Step 1**](#step1), except now patient review embeddings are stored in Neo4j. The chain searches for relevant reviews based on those semantically similar to the user query, and the reviews are used to answer the user query.
* **Wait Times Function**: Similar to the logic in [**Step 1**](#step1), the LangChain agent tries to extract a hospital name from the user query. The hospital name is passed as input to a Python function that gets wait times, and the wait time is returned to the agent.

To walk through an example, suppose a user asks `How many emergency visits were there in 2023?` The LangChain agent will receive this question and decide which tool, if any, to pass the question to. **In this case, the agent should pass the question to the LangChain Neo4j Cypher Chain. The chain will try to convert the question to a Cypher query, run the Cypher query in Neo4j, and use the query results to answer the question**.

Once the LangChain Neo4j Cypher Chain answers the question, it will return the answer to the agent, and the agent will relay the answer to the user.

With this design in mind, you can start building your chatbot. Your first task is to set up a Neo4j AuraDB instance for your chatbot to access.

<a id='step3'></a>
## <b><font color='darkblue'>Step 3: Set Up a Neo4j Graph Database</font></b> ([back](#agenda))
* <font size='3ptx'><b><a href='#step3_1'>A Brief Overview of Graph Databases</a></b></font>
* <font size='3ptx'><b><a href='#step3_2'>Create a Neo4j Account and AuraDB Instance</a></b></font>
* <font size='3ptx'><b><a href='#step3_3'>Design the Hospital System Graph Database</a></b></font>
* <font size='3ptx'><b><a href='#step3_4'>Upload Data to Neo4j</a></b></font>
* <font size='3ptx'><b><a href='#step3_5'>Query the Hospital System Graph</a></b></font>

<b><font size='3ptx'>As you saw in [step 2](#step2), your hospital system data is currently stored in CSV files. Before building your chatbot, you need to store this data in a database that your chatbot can query. You’ll use Neo4j AuraDB for this.</font></b>

Before learning how to set up a Neo4j AuraDB instance, you’ll get an overview of graph databases, and <b>you’ll see why using a graph database may be a better choice than a relational database for this project</b>.

<a id='step3_1'></a>
### <b><font color='darkgreen'>A Brief Overview of Graph Databases</font></b>
<b><font size='3ptx'>Graph databases, such as Neo4j, are databases designed to represent and process data stored as a graph. </font></b>

<b>Graph data consists of nodes, edges or relationships, and properties</b>. Nodes represent entities, relationships connect entities, and properties provide additional metadata about nodes and relationships.

For example, here’s how you might represent hospital system nodes and relationships in a graph:
![hostpital graph](https://files.realpython.com/media/Screenshot_2024-01-16_at_4.33.31_PM.043fc98132e3.png)

This graph has three nodes - <b>Patient</b>, <b>Visit</b>, and <b>Payer</b>. <b>Patient</b> and <b>Visit</b> are connected by the `HAS` relationship, indicating that a hospital patient has a visit. Similarly, <b>Visit</b> and <b>Payer</b> are connected by the `COVERED_BY` relationship, indicating that an insurance payer covers a hospital visit.

<b>Notice how the relationships are represented by an arrow indicating their direction</b>. For example, the direction of the `HAS` relationship tells you that a patient can have a visit, but a visit cannot have a patient.

<b>Both nodes and relationships can have properties.</b> In this example, <b>Patient</b> nodes have `id`, `name`, and `date of birth` properties, and the `COVERED_BY` relationship has `service date` and `billing amount` properties. Storing data in a graph like this has several advantages:
* **Simplicity**: Modeling real-world relationships between entities is natural in graph databases, reducing the need for complex schemas that require multiple join operations to answer queries.
* **Relationships**: Graph databases excel at handling complex relationships. Traversing relationships is efficient, making it easy to query and analyze connected data.
* **Flexibility**: Graph databases are schema-less, allowing for easy adaptation to changing data structures. This flexibility is beneficial for evolving data models.
* **Performance**: Retrieving connected data is faster in graph databases than in relational databases, especially for scenarios involving complex queries with multiple relationships.
* **Pattern Matching**: Graph databases support powerful pattern-matching queries, making it easier to express and find specific structures within the data.

<b>When you have data with many complex relationships, the simplicity and flexibility of graph databases makes them easier to design and query compared to relational databases</b>. As you’ll see later, specifying relationships in graph database queries is concise and doesn’t involve complicated joins. If you’re interested, Neo4j illustrates this well with a realistic example database in their [documentation](https://neo4j.com/developer/cypher/guide-sql-to-cypher/).

Because of this concise data representation, there’s less room for error when an LLM generates graph database queries. This is because you only need to tell the LLM about the nodes, relationships, and properties in your graph database. Contrast this with relational databases where the LLM must navigate and retain knowledge of the table schemas and foreign key relationships throughout your database, leaving more room for error in SQL generation.

<b>Next, you’ll begin working with graph databases by setting up a Neo4j AuraDB instance. After that, you’ll move the hospital system into your Neo4j instance and learn how to query it.</b>

<a id='step3_2'></a>
### <b><font color='darkgreen'>Create a Neo4j Account and AuraDB Instance</font></b> ([back](#step3))
To get started using Neo4j, you can create a free Neo4j AuraDB account. The landing page should look something like this:
![ui](https://files.realpython.com/media/Screenshot_2024-01-10_at_9.52.26_AM.13dfb78c613b.png)
<center>Neo4j Aura getting started screen</center>

<br/>

Click the Start Free button and create an account. Once you’re signed in, you should see the Neo4j Aura console:
![ui](https://files.realpython.com/media/Screenshot_2024-01-10_at_9.53.58_AM.c9d5252982fc.png)
<center>Create a new Aura instance</center>

<br/>

After you click <b>Download and Continue</b>, your instance should be created and a text file containing the Neo4j database credentials should download. Once the instance is created, you’ll see its status is Running. There should be no nodes or relationships yet:
![ui](https://files.realpython.com/media/Screenshot_2024-01-10_at_10.00.34_AM.0ca76879f1fc.png)
<center>Aura running instance</center>

<br/>

Next, open the text file you downloaded with your Neo4j credentials and copy the `NEO4J_URI`, `NEO4J_USERNAME`, and `NEO4J_PASSWORD` into your `.env` file:
* **`.env`**

```shell
OPENAI_API_KEY=<YOUR_OPENAI_API_KEY>

NEO4J_URI=<YOUR_NEO4J_URI>
NEO4J_USERNAME=<YOUR_NEO4J_URI>
NEO4J_PASSWORD=<YOUR_NEO4J_PASSWORD>
```

You’ll use these environment variables to connect to your Neo4j instance in Python so that your chatbot can execute queries.

You now have everything in place to interact with your Neo4j instance. <b>Next up, you’ll design the hospital system graph database. This will tell you how the hospital entities are related, and it will inform the kinds of queries you can run</b>.

<a id='step3_3'></a>
### <b><font color='darkgreen'>Design the Hospital System Graph Database</font></b> ([back](#step3))
<b><font size='3ptx'>Now that you have a running Neo4j AuraDB instance, you need to decide which nodes, relationships, and properties you want to store.</font></b>

One of the most popular ways to represent this is with a flowchart. Based on your understanding of the hospital system data, you come up with the following design:
![flowchart](https://files.realpython.com/media/Screenshot_2024-01-11_at_9.25.30_AM.16896d00ee08.png)
<center>Hospital system graph database design</center>

<br/>

This diagram shows you all of the nodes and relationships in the hospital system data. One useful way to think about this flowchart is to start with the <b>Patient</b> node and follow the relationships. A <b>Patient</b> has a visit at a hospital, and the hospital employs a physician to treat the visit which is covered by an insurance payer.

Here are the properties stored in each node:
![ui](https://files.realpython.com/media/Screenshot_2024-01-17_at_8.28.33_AM.e784ec79aa41.png)
<center>Hospital system node properties</center>

<br/>

The majority of these properties come directly from the fields you explored in [**step 2**](#step2). One notable difference is that <b>Review</b> nodes have an `embedding` property, which is a vector representation of the `patient_name`, `physician_name`, and `text` properties. This allows you to do vector searches over review nodes like you did with ChromaDB.

Here are the relationship properties:
![ui](https://files.realpython.com/media/Screenshot_2024-01-17_at_9.07.16_AM.de07d986e379.png)
<center>Hospital system relationship properties</center>

<br/>

As you can see, `COVERED_BY` is the only relationship with more than an `id` property. The `service_date` is the date the patient was discharged from a visit, and `billing_amount` is the amount charged to the payer for the visit.

<b>Now that you have an overview of the hospital system design you’ll use, it’s time to move your data into Neo4j!</b>

<a id='step3_4'></a>
### <b><font color='darkgreen'>Upload Data to Neo4j</font></b> ([back](#step3))
<b><font size='3ptx'>With a running Neo4j instance and an understanding of the nodes, properties, and relationships you want to store, you can move the hospital system data into Neo4j</font></b>

For this, you’ll create a folder called hospital_neo4j_etl with a few empty files. You’ll also want to create a <font color='olive'>docker-compose.yml</font> file in your project’s root directory:
```
./
│
├── hospital_neo4j_etl/
│   │
│   ├── src/
│   │   ├── entrypoint.sh
│   │   └── hospital_bulk_csv_write.py
│   │
│   ├── Dockerfile
│   └── pyproject.toml
│
├── .env
└── docker-compose.yml
```

Your <font color='olive'>.env</font> file should have the following environment variables:
* `.env.py`

```shell
OPENAI_API_KEY=<YOUR_OPENAI_API_KEY>

NEO4J_URI=<YOUR_NEO4J_URI>
NEO4J_USERNAME=<YOUR_NEO4J_URI>
NEO4J_PASSWORD=<YOUR_NEO4J_PASSWORD>

HOSPITALS_CSV_PATH=https://raw.githubusercontent.com/hfhoffman1144/langchain_neo4j_rag_app/main/data/hospitals.csv
PAYERS_CSV_PATH=https://raw.githubusercontent.com/hfhoffman1144/langchain_neo4j_rag_app/main/data/payers.csv
PHYSICIANS_CSV_PATH=https://raw.githubusercontent.com/hfhoffman1144/langchain_neo4j_rag_app/main/data/physicians.csv
PATIENTS_CSV_PATH=https://raw.githubusercontent.com/hfhoffman1144/langchain_neo4j_rag_app/main/data/patients.csv
VISITS_CSV_PATH=https://raw.githubusercontent.com/hfhoffman1144/langchain_neo4j_rag_app/main/data/visits.csv
REVIEWS_CSV_PATH=https://raw.githubusercontent.com/hfhoffman1144/langchain_neo4j_rag_app/main/data/reviews.csv
```

<b>Notice that you’ve stored all of the CSV files in a public location on GitHub. Because your Neo4j AuraDB instance is running in the cloud, it can’t access files on your local machine, and you have to use HTTP or upload the files directly to your instance</b>. For this example, you can either use the link above, or upload the data to another location.

Once you have your <font color='olive'>.env</font> file populated, open <font color='olive'>pyproject.toml</font>, which provides configuration, metadata, and dependencies defined in the [**TOML**](https://realpython.com/python-toml/) format:
* `hospital_neo4j_etl/pyproject.toml`

```toml
[project]
name = "hospital_neo4j_etl"
version = "0.1"
dependencies = [
   "neo4j==5.14.1",
   "retry==0.9.2"
]

[project.optional-dependencies]
dev = ["black", "flake8"]
```

This project is a bare bones [extract, transform, load (ETL) process](https://en.wikipedia.org/wiki/Extract,_transform,_load) that moves data into Neo4j, so it’s only dependencies are [neo4j](https://pypi.org/project/neo4j/) and [retry](https://pypi.org/project/retry/). The main script for the ETL is <font color='olive'>hospital_neo4j_etl/src/hospital_bulk_csv_write.py</font> ([download](https://drive.google.com/file/d/1j9bWOX9qtUTuGZkCHDRzxRkzuC0RdREA/view?usp=drive_link)). It’s too long to include the full script here, but you’ll get a feel for the main steps. <font color='olive'>hospital_neo4j_etl/src/hospital_bulk_csv_write.py</font> executes. You can copy the full script from the materials.


First, you import dependencies, load environment variables, and configure logging:
* `hospital_neo4j_etl/src/hospital_bulk_csv_write.py`

```python
import os
import logging
from retry import retry
from neo4j import GraphDatabase

HOSPITALS_CSV_PATH = os.getenv("HOSPITALS_CSV_PATH")
PAYERS_CSV_PATH = os.getenv("PAYERS_CSV_PATH")
PHYSICIANS_CSV_PATH = os.getenv("PHYSICIANS_CSV_PATH")
PATIENTS_CSV_PATH = os.getenv("PATIENTS_CSV_PATH")
VISITS_CSV_PATH = os.getenv("VISITS_CSV_PATH")
REVIEWS_CSV_PATH = os.getenv("REVIEWS_CSV_PATH")

NEO4J_URI = os.getenv("NEO4J_URI")
NEO4J_USERNAME = os.getenv("NEO4J_USERNAME")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s]: %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)

LOGGER = logging.getLogger(__name__)

# ...
```

You import the GraphDatabase class from neo4j to connect to your running instance. Notice here that <b>you’re no longer using Python-dotenv to load environment variables. Instead, you’ll pass environment variables into the Docker container that runs your script. Next, you’ll define functions to move hospital data into Neo4j following your design</b>:
* `hospital_neo4j_etl/src/hospital_bulk_csv_write.py`

```python
# ...

NODES = ["Hospital", "Payer", "Physician", "Patient", "Visit", "Review"]

def _set_uniqueness_constraints(tx, node):
    query = f"""CREATE CONSTRAINT IF NOT EXISTS FOR (n:{node})
        REQUIRE n.id IS UNIQUE;"""
    _ = tx.run(query, {})


@retry(tries=100, delay=10)
def load_hospital_graph_from_csv() -> None:
    """Load structured hospital CSV data following
    a specific ontology into Neo4j"""

    driver = GraphDatabase.driver(
        NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD)
    )

    LOGGER.info("Setting uniqueness constraints on nodes")
    with driver.session(database="neo4j") as session:
        for node in NODES:
            session.execute_write(_set_uniqueness_constraints, node)
    # ...

# ...
```

First, you define a helper function, <font color='blue'>_set_uniqueness_constraints()</font>, that creates and runs queries enforcing each node to have a unique ID. In <font color='blue'>load_hospital_graph_from_csv()</font>, you instantiate a driver that connects to your Neo4j instance and set uniqueness constraints for each hospital system node.

Notice the <b><font color='orange'>@retry</font></b> decorator attached <font color='blue'>to load_hospital_graph_from_csv()</font>. If <font color='blue'>to load_hospital_graph_from_csv()</font> fails for any reason, this decorator will rerun it one hundred times with a ten second delay in between tries. This comes in handy when there are intermittent connection issues to Neo4j that are usually resolved by recreating a connection. However, be sure to check the script logs to see if an error reoccurs more than a few times.

Next, <font color='blue'>load_hospital_graph_from_csv()</font> loads data for each node and relationship:
* `hospital_neo4j_etl/src/hospital_bulk_csv_write.py`

```python
# ...

@retry(tries=100, delay=10)
def load_hospital_graph_from_csv() -> None:
    """Load structured hospital CSV data following
    a specific ontology into Neo4j"""

    # ...

    LOGGER.info("Loading hospital nodes")
    with driver.session(database="neo4j") as session:
        query = f"""
        LOAD CSV WITH HEADERS
        FROM '{HOSPITALS_CSV_PATH}' AS hospitals
        MERGE (h:Hospital {{id: toInteger(hospitals.hospital_id),
                            name: hospitals.hospital_name,
                            state_name: hospitals.hospital_state}});
        """
        _ = session.run(query, {})

   # ...

if __name__ == "__main__":
    load_hospital_graph_from_csv()
```

Each node and relationship is loaded from their respective csv files and written to Neo4j according to your graph database design. At the end of the script, you call <font color='blue'>load_hospital_graph_from_csv()</font> in the [name-main idiom](https://realpython.com/if-name-main-python/), and all of the data should populate in your Neo4j instance.

After writing <font color='olive'>hospital_neo4j_etl/src/hospital_bulk_csv_write.py</font>, you can define an <font color='olive'>entrypoint.sh</font> file that will run when your Docker container starts:
* `hospital_neo4j_etl/src/entrypoint.sh`

```shell
#!/bin/bash

# Run any setup steps or pre-processing tasks here
echo "Running ETL to move hospital data from csvs to Neo4j..."

# Run the ETL script
python hospital_bulk_csv_write.py
```

This entrypoint file isn’t technically necessary for this project, but <b>it’s a good practice when building containers because it allows you to execute necessary shell commands before running your main script</b>.

The last file to write for your ETL is the Docker file. It looks like this:
* `hospital_neo4j_etl/Dockerfile`

```yaml
FROM python:3.11-slim

WORKDIR /app

COPY ./src/ /app

COPY ./pyproject.toml /code/pyproject.toml
RUN pip install /code/.

CMD ["sh", "entrypoint.sh"]
```


This <font color='olive'>Dockerfile</font> tells your container to use the `python:3.11-slim distribution`, copy the contents from <font color='olive'>hospital_neo4j_etl/src/</font> into the <font color='olive'>/app</font> directory within the container, install the dependencies from <font color='olive'>pyproject.toml</font>, and run <font color='olive'>entrypoint.sh</font>.

You can now add this project to <font color='olive'>docker-compose.yml</font>:

* `docker-compose.yml`:
```yaml
version: '3'

services:
  hospital_neo4j_etl:
    build:
      context: ./hospital_neo4j_etl
    env_file:
      - .env
```

The ETL will run as a service called `hospital_neo4j_etl`, and it will run the <font color='olive'>Dockerfile</font> in <font color='olive'>./hospital_neo4j_etl</font> using environment variables from <font color='olive'>.env</font>. Since you only have one container, you don’t need docker-compose yet. However, you’ll add more containers to orchestrate with your ETL in the next section, so it’s helpful to get started on <font color='olive'>docker-compose.yml</font>.

To run your ETL, open a terminal and run:
```shell
$ docker-compose up --build
```

Once the ETL finishes running, return to your Aura console:
![ui](https://files.realpython.com/media/Screenshot_2024-01-10_at_10.00.34_AM.0ca76879f1fc.png)
<center>Aura console</center>

<br/>

Click Open and you’ll be prompted to enter your Neo4j password. After successfully logging into the instance, you should see a screen similar to this:
![ui](https://files.realpython.com/media/Screenshot_2024-01-12_at_8.14.38_AM.72233e36a1e0.png)
<center>Neo4j Aura instance with hospital system data loaded</center>

<br/>

As you can see under **Database** Information, all of the nodes, relationships, and properties were loaded. There are 21,187 nodes and 48,259 relationships. You’re ready to start writing queries!

<a id='step3_5'></a>
### <b><font color='darkgreen'>Query the Hospital System Graph</font></b> ([back](#step3))
<b><font size='3ptx'>The last thing you need to do before building your chatbot is get familiar with [Cypher](https://neo4j.com/docs/getting-started/cypher-intro/) syntax.</font></b>

Cypher is Neo4j’s query language, and it’s fairly intuitive to learn, especially if you’re familiar with SQL. This section will cover the basics, and that’s all you need to build the chatbot. You can check out [Neo4j’s documentation](https://neo4j.com/docs/getting-started/cypher-intro/) for a more comprehensive Cypher overview.

TBR (To Be Read)

<a id='step4'></a>
## <b><font color='darkblue'>Step 4: Build a Graph RAG Chatbot in LangChain</font></b> ([back](#agenda))
<b><font size='3ptx'>After all the preparatory design and data work you’ve done so far, you’re finally ready to build your chatbot! </font></b>